In [1]:
!pip install pyarrow
!pip install xgboost lightgbm

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
import copy
from datetime import datetime
import pickle 
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, RepeatedStratifiedKFold, cross_val_score
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb

pd.options.display.max_columns = 200
pd.options.display.max_rows = 200

In [3]:
# df_todo.to_parquet('../tablones/train_data_todo.parquet', engine='pyarrow')
# df_todo_dummies.to_parquet('../tablones/train_data_todo_dummi.parquet', engine='pyarrow')

In [4]:
# Aca solo vamos a trabajar con NUMERICAS, NO DUMMIES
df_todo = pd.read_parquet('../tablones/train_data_todo.parquet', engine='pyarrow')
df_todo.head(1)

,Marca,Cupo,label,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Nr_sum,numero_ventas,Hl_sum,Dcto_sum,sum_2Nr,mean_2Nr,sum_3Nr,mean_3Nr,sum_4Nr,mean_4Nr,sum_5Nr,mean_5Nr,sum_6Nr,mean_6Nr,sum_2Hl,mean_2Hl,sum_3Hl,mean_3Hl,sum_4Hl,mean_4Hl,sum_5Hl,mean_5Hl,sum_6Hl,mean_6Hl,sum_2Dcto,mean_2Dcto,sum_3Dcto,mean_3Dcto,sum_4Dcto,mean_4Dcto,sum_5Dcto,mean_5Dcto,sum_6Dcto,mean_6Dcto,numero_ventas_sum,sum_2numero_ventas,mean_2numero_ventas,sum_3numero_ventas,mean_3numero_ventas,sum_4numero_ventas,mean_4numero_ventas,sum_5numero_ventas,mean_5numero_ventas,sum_6numero_ventas,mean_6numero_ventas,num_clientes_Region,prop_clientes_Region,num_clientes_Gerencia,prop_clientes_Gerencia,num_clientes_SubCanal,prop_clientes_SubCanal,num_clientes_TipoPoblacion,prop_clientes_TipoPoblacion,num_clientes_Estrato,prop_clientes_Estrato,num_clientes_EF,prop_clientes_EF,Num_prom_ofr,ofertas_dadas,ofertas_tomadas,proporcion_tomadas,sum_2ofertas_dadas,mean_2ofertas_dadas,sum_3ofertas_dadas,mean_3ofertas_dadas,sum_4ofertas_dadas,mean_4ofertas_dadas,sum_5ofertas_dadas,mean_5ofertas_dadas,sum_6ofertas_dadas,mean_6ofertas_dadas,sum_2ofertas_tomadas,mean_2ofertas_tomadas,sum_3ofertas_tomadas,mean_3ofertas_tomadas,sum_4ofertas_tomadas,mean_4ofertas_tomadas,sum_5ofertas_tomadas,mean_5ofertas_tomadas,sum_6ofertas_tomadas,mean_6ofertas_tomadas,sum_2proporcion_tomadas,mean_2proporcion_tomadas,sum_3proporcion_tomadas,mean_3proporcion_tomadas,sum_4proporcion_tomadas,mean_4proporcion_tomadas,sum_5proporcion_tomadas,mean_5proporcion_tomadas,sum_6proporcion_tomadas,mean_6proporcion_tomadas
0,29,9,0.0,3,8,15,2,3,1,398013.65,16,1.36,-32036.23,839393.42,419696.71,1248479.17,416159.723333,1785985.13,446496.2825,2127202.34,425440.468,2709897.62,451649.603333,2.75,1.375,4.05,1.35,5.79,1.4475,6.91,1.382,8.9,1.483333,-44248.91,-22124.455,-61301.06,-20433.686667,-92151.35,-23037.8375,-92151.35,-18430.27,-128334.72,-21389.12,16,32.0,16.0,47.0,15.666667,67.0,16.75,81.0,16.2,97.0,16.166667,4981,0.330546,867,0.057535,2293,0.152167,11718,0.777623,4009,0.266043,5405,0.358683,1.0,1,0.0,0.0,6.75,3.375,9.818182,3.272727,12.9,3.225,15.444444,3.088889,18.625,3.104167,0.5,0.25,0.727273,0.242424,0.9,0.225,1.0,0.2,1.125,0.1875,0.062963,0.031481,0.093939,0.031313,0.114444,0.028611,0.12716,0.025432,0.143056,0.023843


# XGBOOST

In [5]:
#train_data, test_data = train_test_split(df_todo_dummies, test_size=0.2)
X_train, X_test, y_train, y_test \
  = train_test_split( df_todo.drop(["label"], axis=1), df_todo["label"], test_size = 0.2)

print(X_train.shape)
print(X_test.shape)

(308641, 99)
(77161, 99)


In [6]:
classifier_xgb = xgb.XGBClassifier(
  n_estimators=100,
  reg_lambda=1,
  gamma=0,
  max_depth=3,
  objective='binary:logistic',
  scale_pos_weight=12.882264042315859,
)
classifier_xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=12.882264042315859,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [7]:
# make predictions for test data    
print("Con scale_pos_weight=12.882264042315859")
predictions = classifier_xgb.predict_proba(X_test)[:, 1]
# evaluate predictions
print('ROC AUC Score',roc_auc_score(y_test, predictions))

Con scale_pos_weight=12.882264042315859
ROC AUC Score 0.9655626221845273


# LIGHTGBM

In [8]:
classifier_lgbm = lgb.LGBMClassifier(
  n_estimators=100,
  learning_rate = 0.05,
  reg_lambda=1,
  max_depth=-1,
  objective='binary',
  scale_pos_weight = 12.882264042315859,
)
classifier_lgbm.fit(X_train, y_train, eval_metric='auc')

LGBMClassifier(learning_rate=0.05, objective='binary', reg_lambda=1,
               scale_pos_weight=12.882264042315859)

In [9]:
# make predictions for test data    
predictions = classifier_lgbm.predict_proba(X_test)[:, 1]
# evaluate predictions
print('ROC AUC Score',roc_auc_score(y_test, predictions))
#ROC AUC Score 0.8932139912379119

ROC AUC Score 0.9665889922395161


# PREDECIR EN EL TEST

In [10]:
df_test = pd.read_csv('../data/test.csv', encoding='ISO-8859-1', sep=',')
df_test.head(1)

,Cliente,Marca,Cupo,Ejecuto_Promo
0,10,40,16,NaN


## agregar fecha para cortes

In [11]:
# Agregar month_year = '201908', se asume
df_test['month_year'] = pd.Period('2019-08', 'M')
# Dropear Ejecuto_Promo, al final lo agrego otra vez
df_test = df_test.drop('Ejecuto_Promo', axis = 1)
print(df_test.isna().sum().sum())
display(df_test.head(1))

0


,Cliente,Marca,Cupo,month_year
0,10,40,16,2019-08


## agregar data de clientes sin enriquecer

In [12]:
df_clients = pd.read_csv('../data/clients_attributes.csv', encoding='ISO-8859-1', sep=',')
# Agregamos informacion de clientes sin enriquecer
df_t1 = pd.merge(
  df_test,
  df_clients[['Cliente', 'Region', 'Gerencia', 'SubCanal', 'TipoPoblacion', 'Estrato', 'EF']],
  on = 'Cliente',
  how = 'left'
)
print(df_t1.isna().sum().sum())
display(df_t1.head(1))

0


,Cliente,Marca,Cupo,month_year,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF
0,10,40,16,2019-08,4,21,15,2,2,0


## agregamos data de ventas enriquecida

In [13]:
# Cargamos pickle
f_ventas = '../tablones/data_grupo_3.pkl'
def load_pickle(fp):
  with open(fp, 'rb') as f:
    df = pickle.load(f)
  return df
df_ventas_enriched = load_pickle(f_ventas)
print(df_ventas_enriched.isna().sum().sum())
# Creamos month_year
df_ventas_enriched['month_year'] = df_ventas_enriched['periodo'].dt.to_period('M')
df_ventas_enriched = df_ventas_enriched.drop('periodo', axis=1)
display(df_ventas_enriched.head(1))

0


,Cliente,Nr_sum,numero_ventas,Hl_sum,Dcto_sum,sum_2Nr,mean_2Nr,sum_3Nr,mean_3Nr,sum_4Nr,mean_4Nr,sum_5Nr,mean_5Nr,sum_6Nr,mean_6Nr,sum_2Hl,mean_2Hl,sum_3Hl,mean_3Hl,sum_4Hl,mean_4Hl,sum_5Hl,mean_5Hl,sum_6Hl,mean_6Hl,sum_2Dcto,mean_2Dcto,sum_3Dcto,mean_3Dcto,sum_4Dcto,mean_4Dcto,sum_5Dcto,mean_5Dcto,sum_6Dcto,mean_6Dcto,numero_ventas_sum,sum_2numero_ventas,mean_2numero_ventas,sum_3numero_ventas,mean_3numero_ventas,sum_4numero_ventas,mean_4numero_ventas,sum_5numero_ventas,mean_5numero_ventas,sum_6numero_ventas,mean_6numero_ventas,month_year
0,1,591975.69,13,2.31,-6785.54,680049.570526,340024.785263,1032291.12,344097.04,1.389348e+06,347337.026471,1.764272e+06,352854.43425,2.180481e+06,363413.575889,2.491579,1.245789,3.772778,1.257593,5.072353,1.268088,6.429375,1.285875,7.932,1.322,-47691.214211,-23845.607105,-72644.81,-24214.936667,-98924.870588,-24731.217647,-127734.36625,-25546.87325,-160123.783333,-26687.297222,13,17.736842,8.868421,26.722222,8.907407,35.882353,8.970588,45.0625,9.0125,54.933333,9.155556,2018-01


In [14]:
# Convertir fechas a string para facilitar cruce
df_t1['month_year'] = df_t1['month_year'].astype(str)
df_ventas_enriched['month_year'] = df_ventas_enriched['month_year'].astype(str)


In [15]:
# Agregamos informacion de ventas enriquecida
df_t2 = pd.merge(
  df_t1,
  df_ventas_enriched,
  on = ['month_year', 'Cliente'],
  how = 'left'  
)
print(df_t2.isna().sum().sum())
display(df_t2.head(1))

0


,Cliente,Marca,Cupo,month_year,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Nr_sum,numero_ventas,Hl_sum,Dcto_sum,sum_2Nr,mean_2Nr,sum_3Nr,mean_3Nr,sum_4Nr,mean_4Nr,sum_5Nr,mean_5Nr,sum_6Nr,mean_6Nr,sum_2Hl,mean_2Hl,sum_3Hl,mean_3Hl,sum_4Hl,mean_4Hl,sum_5Hl,mean_5Hl,sum_6Hl,mean_6Hl,sum_2Dcto,mean_2Dcto,sum_3Dcto,mean_3Dcto,sum_4Dcto,mean_4Dcto,sum_5Dcto,mean_5Dcto,sum_6Dcto,mean_6Dcto,numero_ventas_sum,sum_2numero_ventas,mean_2numero_ventas,sum_3numero_ventas,mean_3numero_ventas,sum_4numero_ventas,mean_4numero_ventas,sum_5numero_ventas,mean_5numero_ventas,sum_6numero_ventas,mean_6numero_ventas
0,10,40,16,2019-08,4,21,15,2,2,0,818199.93,14,3.58,-43358.46,1521431.02,760715.51,2897990.58,965996.86,3580176.93,895044.2325,4375014.64,875002.928,5271955.73,878659.288333,6.66,3.33,12.19,4.063333,14.78,3.695,18.0,3.6,21.67,3.611667,-62775.26,-31387.63,-133840.46,-44613.486667,-161979.51,-40494.8775,-199488.19,-39897.638,-262790.86,-43798.476667,14,29.0,14.5,41.0,13.666667,50.0,12.5,63.0,12.6,81.0,13.5


## agregamos data de proporciones de clientes

In [16]:
# Agregamos informacion de prop_num 
# ESTO SE DEBERIA IMPORTAR DE UN UTILITARIO
def obtener_prop_num(df, col_variable):
  """
  df: pd.Dataframe
  col_variable: str
  """
  # Copia: para asegurarnos que no modificamos df -> se debe hacer refactoring
  c_df = copy.deepcopy(df)
  # Cuenta de registros en tabla de clientes
  numero_clientes = len(c_df)
  
  # Contamos registros por valor de col_variable
  df_g1 = c_df.groupby([col_variable]).agg({
    'FechaAltaCliente': 'count',
  }).reset_index()
  
  # Creamos nombre de la columna donde se almacena el numero de clientes
  col_num_clientes = 'num_clientes_' + col_variable
  # Renombrar columna donde se almacena el numero de clientes
  df_g1 = df_g1.rename(columns={'FechaAltaCliente': col_num_clientes})
  # Creamos nombre de la columna donde se almacena la proporcion de clientes
  col_proporcion_clientes = 'prop_clientes_' + col_variable
  # Agregamos columna que almacena informacion de propocion de clientes 
  df_g1[col_proporcion_clientes] = df_g1[col_num_clientes] / numero_clientes
  
  return df_g1

cols_cat_clientes = ['Region', 'Gerencia', 'SubCanal', 'TipoPoblacion', 'Estrato', 'EF']
df_t3 = copy.deepcopy(df_t2)
for col_name in cols_cat_clientes:
  curr_df = obtener_prop_num(df_clients, col_name) 
  df_t3 = pd.merge(df_t3, curr_df, how = 'left', on = col_name)
  #print(df_t3.isna().sum().sum())
print(df_t3.isna().sum().sum())
display(df_t3.head(1))  

0


,Cliente,Marca,Cupo,month_year,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Nr_sum,numero_ventas,Hl_sum,Dcto_sum,sum_2Nr,mean_2Nr,sum_3Nr,mean_3Nr,sum_4Nr,mean_4Nr,sum_5Nr,mean_5Nr,sum_6Nr,mean_6Nr,sum_2Hl,mean_2Hl,sum_3Hl,mean_3Hl,sum_4Hl,mean_4Hl,sum_5Hl,mean_5Hl,sum_6Hl,mean_6Hl,sum_2Dcto,mean_2Dcto,sum_3Dcto,mean_3Dcto,sum_4Dcto,mean_4Dcto,sum_5Dcto,mean_5Dcto,sum_6Dcto,mean_6Dcto,numero_ventas_sum,sum_2numero_ventas,mean_2numero_ventas,sum_3numero_ventas,mean_3numero_ventas,sum_4numero_ventas,mean_4numero_ventas,sum_5numero_ventas,mean_5numero_ventas,sum_6numero_ventas,mean_6numero_ventas,num_clientes_Region,prop_clientes_Region,num_clientes_Gerencia,prop_clientes_Gerencia,num_clientes_SubCanal,prop_clientes_SubCanal,num_clientes_TipoPoblacion,prop_clientes_TipoPoblacion,num_clientes_Estrato,prop_clientes_Estrato,num_clientes_EF,prop_clientes_EF
0,10,40,16,2019-08,4,21,15,2,2,0,818199.93,14,3.58,-43358.46,1521431.02,760715.51,2897990.58,965996.86,3580176.93,895044.2325,4375014.64,875002.928,5271955.73,878659.288333,6.66,3.33,12.19,4.063333,14.78,3.695,18.0,3.6,21.67,3.611667,-62775.26,-31387.63,-133840.46,-44613.486667,-161979.51,-40494.8775,-199488.19,-39897.638,-262790.86,-43798.476667,14,29.0,14.5,41.0,13.666667,50.0,12.5,63.0,12.6,81.0,13.5,3531,0.234322,434,0.028801,2293,0.152167,11718,0.777623,6598,0.437853,9664,0.641317


## agregamos data de promociones activas y ejecutdas enriquecida

In [18]:
f_prop = '../tablones/data_grupo2_proporcion.pkl' 
df_prop_enriched = load_pickle(f_prop)
df_prop_enriched.head(1)
df_prop_enriched['month_year'] = df_prop_enriched['month_year'].astype(str)
# Cruzamos df_prop_enriched(enriched) 
df_t4 = pd.merge(
  df_t3,
  df_prop_enriched,
  on = ['month_year', 'Cliente'],
  how = 'left'
)
print(df_t4.isna().sum()/len(df_t4)*100)
print(df_t4.isna().sum().sum())
display(df_t4.head(1))

Cliente                        0.0
Marca                          0.0
Cupo                           0.0
month_year                     0.0
Region                         0.0
Gerencia                       0.0
SubCanal                       0.0
TipoPoblacion                  0.0
Estrato                        0.0
EF                             0.0
Nr_sum                         0.0
numero_ventas                  0.0
Hl_sum                         0.0
Dcto_sum                       0.0
sum_2Nr                        0.0
mean_2Nr                       0.0
sum_3Nr                        0.0
mean_3Nr                       0.0
sum_4Nr                        0.0
mean_4Nr                       0.0
sum_5Nr                        0.0
mean_5Nr                       0.0
sum_6Nr                        0.0
mean_6Nr                       0.0
sum_2Hl                        0.0
mean_2Hl                       0.0
sum_3Hl                        0.0
mean_3Hl                       0.0
sum_4Hl             

,Cliente,Marca,Cupo,month_year,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Nr_sum,numero_ventas,Hl_sum,Dcto_sum,sum_2Nr,mean_2Nr,sum_3Nr,mean_3Nr,sum_4Nr,mean_4Nr,sum_5Nr,mean_5Nr,sum_6Nr,mean_6Nr,sum_2Hl,mean_2Hl,sum_3Hl,mean_3Hl,sum_4Hl,mean_4Hl,sum_5Hl,mean_5Hl,sum_6Hl,mean_6Hl,sum_2Dcto,mean_2Dcto,sum_3Dcto,mean_3Dcto,sum_4Dcto,mean_4Dcto,sum_5Dcto,mean_5Dcto,sum_6Dcto,mean_6Dcto,numero_ventas_sum,sum_2numero_ventas,mean_2numero_ventas,sum_3numero_ventas,mean_3numero_ventas,sum_4numero_ventas,mean_4numero_ventas,sum_5numero_ventas,mean_5numero_ventas,sum_6numero_ventas,mean_6numero_ventas,num_clientes_Region,prop_clientes_Region,num_clientes_Gerencia,prop_clientes_Gerencia,num_clientes_SubCanal,prop_clientes_SubCanal,num_clientes_TipoPoblacion,prop_clientes_TipoPoblacion,num_clientes_Estrato,prop_clientes_Estrato,num_clientes_EF,prop_clientes_EF,ofertas_dadas,ofertas_tomadas,proporcion_tomadas,sum_2ofertas_dadas,mean_2ofertas_dadas,sum_3ofertas_dadas,mean_3ofertas_dadas,sum_4ofertas_dadas,mean_4ofertas_dadas,sum_5ofertas_dadas,mean_5ofertas_dadas,sum_6ofertas_dadas,mean_6ofertas_dadas,sum_2ofertas_tomadas,mean_2ofertas_tomadas,sum_3ofertas_tomadas,mean_3ofertas_tomadas,sum_4ofertas_tomadas,mean_4ofertas_tomadas,sum_5ofertas_tomadas,mean_5ofertas_tomadas,sum_6ofertas_tomadas,mean_6ofertas_tomadas,sum_2proporcion_tomadas,mean_2proporcion_tomadas,sum_3proporcion_tomadas,mean_3proporcion_tomadas,sum_4proporcion_tomadas,mean_4proporcion_tomadas,sum_5proporcion_tomadas,mean_5proporcion_tomadas,sum_6proporcion_tomadas,mean_6proporcion_tomadas
0,10,40,16,2019-08,4,21,15,2,2,0,818199.93,14,3.58,-43358.46,1521431.02,760715.51,2897990.58,965996.86,3580176.93,895044.2325,4375014.64,875002.928,5271955.73,878659.288333,6.66,3.33,12.19,4.063333,14.78,3.695,18.0,3.6,21.67,3.611667,-62775.26,-31387.63,-133840.46,-44613.486667,-161979.51,-40494.8775,-199488.19,-39897.638,-262790.86,-43798.476667,14,29.0,14.5,41.0,13.666667,50.0,12.5,63.0,12.6,81.0,13.5,3531,0.234322,434,0.028801,2293,0.152167,11718,0.777623,6598,0.437853,9664,0.641317,5,1.0,0.2,8.0,4.0,8.0,2.666667,10.0,2.5,12.0,2.4,16.0,2.666667,2.0,1.0,2.0,0.666667,2.0,0.5,2.0,0.4,2.0,0.333333,0.533333,0.266667,0.533333,0.177778,0.533333,0.133333,0.533333,0.106667,0.533333,0.088889


## agregamos data de numero producto ofrecido

In [19]:
# Cargar data de prom ofrecidas
# Arreglar fecha para poder cruzar sin problemas
# Quitar registros correspondientes a septiembre 2019
# Conversion de tipos para poder cruzar
f_promo_ofrecidas = '../tablones/num_prom_ofr.pkl'
def load_pickle(fp):
  with open(fp, 'rb') as f:
    df = pickle.load(f)
  return df
df_promo_ofrecidas = load_pickle(f_promo_ofrecidas)
df_promo_ofrecidas['Date_Desde'] \
  = pd.to_datetime(df_promo_ofrecidas['Date_Desde'], format= "%Y-%m")
# Convertir Date_Desde a periodo -> month_year
df_promo_ofrecidas['month_year'] = df_promo_ofrecidas['Date_Desde'].dt.to_period('M')
# Dropear columnas innecesarias
df_promo_ofrecidas = df_promo_ofrecidas.drop('Date_Desde', axis = 1)
# QUITAR DATA DE SEPTIEMBRE PORQUE ESTO NO SE USA PARA ENTRENAR POR VARIAS RAZONES
df_promo_ofrecidas = df_promo_ofrecidas[df_promo_ofrecidas['month_year'] != '201909']
# Convertir columnas a int para poder cruzar
df_promo_ofrecidas['Marca'] = df_promo_ofrecidas['Marca'].astype(int)
df_promo_ofrecidas['Cupo'] = df_promo_ofrecidas['Cupo'].astype(int)
df_promo_ofrecidas['Cliente'] = df_promo_ofrecidas['Cliente'].astype(int)
# convertimos month_year en str
df_promo_ofrecidas['month_year'] = df_promo_ofrecidas['month_year'].astype(str)
df_promo_ofrecidas.head(1)

,Marca,Cupo,Cliente,Num_prom_ofr,month_year
0,29,9,10358,1,2018-08


In [20]:
df_promo_ofrecidas['month_year'].unique()

array(['2018-08', '2018-09', '2018-10', '2018-11', '2018-12', '2019-01',
       '2019-02', '2019-03', '2019-04', '2019-05', '2019-06', '2019-07',
       '2019-08'], dtype=object)

In [21]:
# Agregar info de promo ofrecidas
df_t5 = pd.merge(
  df_t4,
  df_promo_ofrecidas,
  on = ['month_year', 'Cliente', 'Marca', 'Cupo'],
  how = 'left'
)
print(df_t5.isna().sum().sum())
display(df_t5.head(1))  

1025


,Cliente,Marca,Cupo,month_year,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Nr_sum,numero_ventas,Hl_sum,Dcto_sum,sum_2Nr,mean_2Nr,sum_3Nr,mean_3Nr,sum_4Nr,mean_4Nr,sum_5Nr,mean_5Nr,sum_6Nr,mean_6Nr,sum_2Hl,mean_2Hl,sum_3Hl,mean_3Hl,sum_4Hl,mean_4Hl,sum_5Hl,mean_5Hl,sum_6Hl,mean_6Hl,sum_2Dcto,mean_2Dcto,sum_3Dcto,mean_3Dcto,sum_4Dcto,mean_4Dcto,sum_5Dcto,mean_5Dcto,sum_6Dcto,mean_6Dcto,numero_ventas_sum,sum_2numero_ventas,mean_2numero_ventas,sum_3numero_ventas,mean_3numero_ventas,sum_4numero_ventas,mean_4numero_ventas,sum_5numero_ventas,mean_5numero_ventas,sum_6numero_ventas,mean_6numero_ventas,num_clientes_Region,prop_clientes_Region,num_clientes_Gerencia,prop_clientes_Gerencia,num_clientes_SubCanal,prop_clientes_SubCanal,num_clientes_TipoPoblacion,prop_clientes_TipoPoblacion,num_clientes_Estrato,prop_clientes_Estrato,num_clientes_EF,prop_clientes_EF,ofertas_dadas,ofertas_tomadas,proporcion_tomadas,sum_2ofertas_dadas,mean_2ofertas_dadas,sum_3ofertas_dadas,mean_3ofertas_dadas,sum_4ofertas_dadas,mean_4ofertas_dadas,sum_5ofertas_dadas,mean_5ofertas_dadas,sum_6ofertas_dadas,mean_6ofertas_dadas,sum_2ofertas_tomadas,mean_2ofertas_tomadas,sum_3ofertas_tomadas,mean_3ofertas_tomadas,sum_4ofertas_tomadas,mean_4ofertas_tomadas,sum_5ofertas_tomadas,mean_5ofertas_tomadas,sum_6ofertas_tomadas,mean_6ofertas_tomadas,sum_2proporcion_tomadas,mean_2proporcion_tomadas,sum_3proporcion_tomadas,mean_3proporcion_tomadas,sum_4proporcion_tomadas,mean_4proporcion_tomadas,sum_5proporcion_tomadas,mean_5proporcion_tomadas,sum_6proporcion_tomadas,mean_6proporcion_tomadas,Num_prom_ofr
0,10,40,16,2019-08,4,21,15,2,2,0,818199.93,14,3.58,-43358.46,1521431.02,760715.51,2897990.58,965996.86,3580176.93,895044.2325,4375014.64,875002.928,5271955.73,878659.288333,6.66,3.33,12.19,4.063333,14.78,3.695,18.0,3.6,21.67,3.611667,-62775.26,-31387.63,-133840.46,-44613.486667,-161979.51,-40494.8775,-199488.19,-39897.638,-262790.86,-43798.476667,14,29.0,14.5,41.0,13.666667,50.0,12.5,63.0,12.6,81.0,13.5,3531,0.234322,434,0.028801,2293,0.152167,11718,0.777623,6598,0.437853,9664,0.641317,5,1.0,0.2,8.0,4.0,8.0,2.666667,10.0,2.5,12.0,2.4,16.0,2.666667,2.0,1.0,2.0,0.666667,2.0,0.5,2.0,0.4,2.0,0.333333,0.533333,0.266667,0.533333,0.177778,0.533333,0.133333,0.533333,0.106667,0.533333,0.088889,2.0


In [22]:
# Llena con 0 los NaN en Num_prom_ofr porque significa que no se ofrecieron descuentos
df_t5['Num_prom_ofr'] = df_t5['Num_prom_ofr'].fillna(0)
print(df_t5.isna().sum().sum())
display(df_t5.head(1))  

0


,Cliente,Marca,Cupo,month_year,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Nr_sum,numero_ventas,Hl_sum,Dcto_sum,sum_2Nr,mean_2Nr,sum_3Nr,mean_3Nr,sum_4Nr,mean_4Nr,sum_5Nr,mean_5Nr,sum_6Nr,mean_6Nr,sum_2Hl,mean_2Hl,sum_3Hl,mean_3Hl,sum_4Hl,mean_4Hl,sum_5Hl,mean_5Hl,sum_6Hl,mean_6Hl,sum_2Dcto,mean_2Dcto,sum_3Dcto,mean_3Dcto,sum_4Dcto,mean_4Dcto,sum_5Dcto,mean_5Dcto,sum_6Dcto,mean_6Dcto,numero_ventas_sum,sum_2numero_ventas,mean_2numero_ventas,sum_3numero_ventas,mean_3numero_ventas,sum_4numero_ventas,mean_4numero_ventas,sum_5numero_ventas,mean_5numero_ventas,sum_6numero_ventas,mean_6numero_ventas,num_clientes_Region,prop_clientes_Region,num_clientes_Gerencia,prop_clientes_Gerencia,num_clientes_SubCanal,prop_clientes_SubCanal,num_clientes_TipoPoblacion,prop_clientes_TipoPoblacion,num_clientes_Estrato,prop_clientes_Estrato,num_clientes_EF,prop_clientes_EF,ofertas_dadas,ofertas_tomadas,proporcion_tomadas,sum_2ofertas_dadas,mean_2ofertas_dadas,sum_3ofertas_dadas,mean_3ofertas_dadas,sum_4ofertas_dadas,mean_4ofertas_dadas,sum_5ofertas_dadas,mean_5ofertas_dadas,sum_6ofertas_dadas,mean_6ofertas_dadas,sum_2ofertas_tomadas,mean_2ofertas_tomadas,sum_3ofertas_tomadas,mean_3ofertas_tomadas,sum_4ofertas_tomadas,mean_4ofertas_tomadas,sum_5ofertas_tomadas,mean_5ofertas_tomadas,sum_6ofertas_tomadas,mean_6ofertas_tomadas,sum_2proporcion_tomadas,mean_2proporcion_tomadas,sum_3proporcion_tomadas,mean_3proporcion_tomadas,sum_4proporcion_tomadas,mean_4proporcion_tomadas,sum_5proporcion_tomadas,mean_5proporcion_tomadas,sum_6proporcion_tomadas,mean_6proporcion_tomadas,Num_prom_ofr
0,10,40,16,2019-08,4,21,15,2,2,0,818199.93,14,3.58,-43358.46,1521431.02,760715.51,2897990.58,965996.86,3580176.93,895044.2325,4375014.64,875002.928,5271955.73,878659.288333,6.66,3.33,12.19,4.063333,14.78,3.695,18.0,3.6,21.67,3.611667,-62775.26,-31387.63,-133840.46,-44613.486667,-161979.51,-40494.8775,-199488.19,-39897.638,-262790.86,-43798.476667,14,29.0,14.5,41.0,13.666667,50.0,12.5,63.0,12.6,81.0,13.5,3531,0.234322,434,0.028801,2293,0.152167,11718,0.777623,6598,0.437853,9664,0.641317,5,1.0,0.2,8.0,4.0,8.0,2.666667,10.0,2.5,12.0,2.4,16.0,2.666667,2.0,1.0,2.0,0.666667,2.0,0.5,2.0,0.4,2.0,0.333333,0.533333,0.266667,0.533333,0.177778,0.533333,0.133333,0.533333,0.106667,0.533333,0.088889,2.0


## Revisar que las columnas coincidan con lo que se utilizo para entrar

In [23]:
dif_set = set(X_train.columns).difference(set(df_t5.columns))
print(dif_set)

set()


In [24]:
dif_2 = list(set(df_t5.columns).difference(set(X_train.columns)))
print(dif_2)

['Cliente', 'month_year']


In [25]:
#df_t5_dummies = df_t5_dummies.drop('Cliente', axis=1)
df_t5 = df_t5.drop(columns = dif_2, axis = 1)

In [26]:
set(X_train.columns).difference(set(df_t5.columns))

set()

In [27]:
list(set(df_t5.columns).difference(set(X_train.columns)))

[]

# Prediccion

In [28]:
# Hacemos la prediccion con el modelo que se Entreno con solo NUMERICAS, NOO DUMMIES
modelo = classifier_lgbm 
pred = modelo.predict_proba(df_t5)
print(pred)

[[0.00780762 0.99219238]
 [0.00617109 0.99382891]
 [0.99668746 0.00331254]
 ...
 [0.99672365 0.00327635]
 [0.99668746 0.00331254]
 [0.99668746 0.00331254]]


In [29]:
# Obtenemos las probabilidades de que la categoria = 1
lista_pred_compro = [x for _, x in pred]
len(lista_pred_compro)

16870

In [33]:
# Cargas el .csv de output
df_output = pd.read_csv('../data/test.csv', encoding='ISO-8859-1', sep=',')
df_output.tail(1)

,Cliente,Marca,Cupo,Ejecuto_Promo
16869,15064,29,9,NaN


In [34]:
# Agregamos nuestras predicciones al test.csv
df_output['Ejecuto_Promo'] = lista_pred_compro
print(df_output.isna().sum().sum())
display(df_output.head(1))

0


,Cliente,Marca,Cupo,Ejecuto_Promo
0,10,40,16,0.992192


In [37]:
import time
# Creamos el nombre para nuestro archivo usando la fecha actual
fp = '../output/' + str(time.asctime()) + '.csv'
print(fp)

../output/Sun Nov 29 05:39:48 2020.csv


In [38]:
# Almacenamos sin index
df_output.to_csv(fp, sep = ',', columns = ['Cliente', 'Marca', 'Cupo', 'Ejecuto_Promo'], index=False)